# Создал этот ноутбук для проверки простеньких сеток на синтетических датасетах / кусках датасетов

- Векторы V все почти что коллинеарны, когда их много - стоит уже посмотреть на $p_i$ и $r_{cut_{i}}$

- 

In [106]:
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import torch.optim as optim
from torch.optim import lr_scheduler

import seaborn as sns
sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})

In [107]:
def function(X):
    return 5 * X

In [108]:
path = '3_dataset_K_3.pt'

In [109]:
class CFG:
    '''

    All hyperparameters are here

    '''

    N = int(path.split('_')[0])     # число атомов
    K = int(path.split('_')[-1].split('.')[0])     # можно называть это разрешением...чем число больше, тем больше размеры матрицы для атомов, фактически это число элементов в наборах p и r_cut

    L = 2 * N ** (1 / 3) # размер одной клетки при моделировании

    r_cut = np.random.uniform(low=5, high=10, size=K).copy()
    p = np.random.uniform(low=1, high=3, size=K).copy()
    N_neig= N - 1 if N != 2 else 1

    # train_bs = 8
    # val_bs = 16
    batch_size = 64

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    f_threshold = 5    # Если сила по какой-то координате превышает это значение, то строчка исключается, совсем маленьких по модулю сил быть не должно, если что при генерации просто r_cut поменьше надо делать
    coord_threshold = L     # Если вдруг очень большие расстояния, то надо выкидывать
    f_min_threshold = 0.05
    #
    output_size = K     # Размерность аутпута модели

In [110]:
def create_dataloaders(train_dataset, val_dataset, train_bs=64, val_bs=64, fold=None):
    '''

    Returns train_loader, val_loader

    fold: will be used in cross validation, when I will implement it

    '''
    
    train_loader = DataLoader(dataset=train_dataset, batch_size=train_bs, shuffle=True)

    val_loader = DataLoader(dataset=val_dataset, batch_size=val_bs, shuffle=False)

    return train_loader, val_loader

In [111]:
def plot_k_dim_f_norm_from_x_norm(data_sample, figsize=(15, 10), fontsize=18):
    '''

    Функция строит зависимость таргета от X

    data_sample - итерируемый объект из тьюплов вида: (x, f_k_dim, f_3d, pinv_A)

    '''
    plt.figure(figsize=figsize)
    x = torch.stack([elem[0][0].squeeze() for elem in data_sample])

    y = torch.stack([elem[1][0].squeeze() for elem in data_sample])

    plt.scatter(x, y, label='y(x)')
    plt.title('Зависимость $Y_{target}(X)$', fontsize=fontsize)

    None

In [112]:
# class MultiOutputMSELoss(nn.MSELoss):
#     '''

#     Custom loss that calculates average over batch loss for multidim MSE - sum of MSE for components

#     Example:
#     |
#     |    Loss = MultiOutputMSELoss()
#     |
#     |    a = torch.ones((8, 3))      # it is batch of 8 vectors of size 3
#     |    b = torch.zeros((8, 3))
#     |
#     |    Loss(a, b, batch_size=8) -> 3

#     '''

#     def forward(self, input, target, batch_size=CFG.batch_size):
#         '''
#         оно при reduction='mean' делит на произведение всех размерностей
#         '''
#         # при очень большом размере батча последние батчи будут например размера 128 вместо 256, поэтому просто умножать на батч сайз неправильно, могут быть другого размера

#         return F.mse_loss(input, target, reduction='sum') / input.size(0)   # или эквивалентно делать reduction='mean' и умножать на input.size()[-1] - length of output

In [113]:
def make_predictions_and_plot(model, X_matrices=None, Y_target=None, figsize=(30, 20), fontsize=20, criterion=nn.MSELoss(), data=None, same_axis=False):
    '''

    Строит предсказанную и тагрет зависимости

    Можно подавать либо отлельно X_matrices, Y_target либо подать датасет из тьюплов: (X, f_k_dim, f_3d, A_pinv)

    Будет подаваться 3 итерируемых объекта: набор(батч или кусок датасета) из "матриц", предсказания, таргеты

    Качество

    '''

    if data:
        X_matrices, Y_target = list(map(lambda x: torch.stack(x), list(zip(*data))))
        
    Y_pred = model(X_matrices)
    metric = criterion(Y_pred, Y_target)
    names = ['predicted', 'target']
    Ys = [Y_pred, Y_target]
    metric = criterion(Y_pred, Y_target)
    
    if CFG.K != 1:
        print(
            f'Metric: {metric}'
        )
        return

    plt.figure(figsize=figsize)
    fig = plt.gcf()
    fig.suptitle(f'Metric: {metric}', fontsize=22)

    for (i, name) in enumerate(names):
        if not same_axis:
            plt.subplot(1, 2, i + 1)

        plt.scatter(X_matrices.squeeze().detach().numpy(), Ys[i].squeeze().detach().numpy(), label='Y(X)')

        plt.xlabel('X', fontsize=18)
        plt.ylabel('Y', fontsize=18)
        plt.legend(loc='best', fontsize=fontsize)
        plt.title(f'{name}', fontsize=18)
        plt.grid(alpha=0.4)

    plt.show()

In [114]:
def fit_epoch(model, train_loader, criterion, optimizer, scheduler=None, scaler=None):
    '''

    Функция обучения по всем батчам 1 раз (1 эпоха)

    scaler: gradient scaler from torch.amp, попозже добавлю обучение с ним

    В данной версии: (X, f_k_dim)

    Лосс выводится для k-мерного предсказания, а mse считается по 3D вариантам, однако при K=1 3d и 1d MSE совпадают

    '''
    model.train()

    running_loss = 0.0
    # running_MSE = 0
    processed_data = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()

        inputs = inputs.to(CFG.device)
        labels = labels.to(CFG.device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        if scheduler:
            scheduler.step()

        running_loss += loss.item() * inputs.size(0)    # при очень большом размере батча последние два батча будут например размера 128 вместо 256, поэтому просто умножать на батч сайз неправильно, могут быть другого размера
        processed_data += inputs.size(0)

    # print(labels)
    train_loss = running_loss / processed_data
    # train_MSE = running_MSE / processed_data
    
    return train_loss

In [115]:
def eval_epoch(model, val_loader, criterion):
    '''

    Одна эпоха по val выборке

    '''

    model.eval()
    
    running_loss = 0.0
    # running_MSE = 0
    processed_size = 0

    for inputs, labels in val_loader:

        inputs = inputs.to(CFG.device)
        labels = labels.to(CFG.device)

        with torch.no_grad():
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        running_loss += loss.item() * inputs.size(0)
        processed_size += inputs.size(0)

    # print(f' outputs:\n{outputs}, \n labels: \n {labels}')
    
    val_loss = running_loss / processed_size
    # val_MSE = running_MSE / processed_size

    return val_loss

In [116]:
def train(train_loader, val_loader, model, optimizer, scheduler=None, epochs=10, scaler=None, criterion=nn.MSELoss()):
    '''

    Basic option: calculation loss on K-dimensional outputs, but MSE metric on 3D outputs, after the matrix is applied

    loss_on_k_projections: calculate loss'
    
    '''

    history = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} val_loss {v_loss:0.4f}"

    with tqdm(desc="epoch", total=epochs) as pbar_outer:

        for epoch in range(epochs):
            train_loss = fit_epoch(model, train_loader, criterion, optimizer, scheduler, scaler)

            val_loss = eval_epoch(model, val_loader, criterion)
            
            history.append((train_loss, val_loss))
            
            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch + 1, t_loss=train_loss, v_loss=val_loss))
            
    return history

In [117]:
class SingleNet(nn.Module):
    '''

    Класс одиночной нейронной сети

    '''
    def __init__(self, output_size, activation=nn.ReLU(), flattened_size=CFG.K * CFG.K):
        '''
        
        FC_type: тип полносвязных слоев: 'regular' / 'simple

        convolution: сверточная часть сети

        '''
        super().__init__()

        self.FC = nn.Sequential(
            nn.Linear(flattened_size, 64),
            activation,
            # nn.Dropout(0.3),
            nn.BatchNorm1d(64),

            nn.Linear(64, 128),
            activation,
            # nn.Dropout(0.3),
            nn.BatchNorm1d(128),
            
            nn.Linear(128, 256),
            activation,
            # nn.Dropout(0.3),
            nn.BatchNorm1d(256),
            nn.Linear(256, output_size),
        )

        # self.FC = nn.Linear(flattened_size, output_size)

    def forward(self, x):
        # x - is batch of matrices KxK

        # Здесь происходят какие-то там свертки, пуллинги и тп..

        x = self.FC(x)

        return x

In [161]:
def recieve_loaders(take_one_projection_for_data=None, path=None, cut_size=None):
    '''
    returns: (dataet, train_loader, val_loader), if u pass path -> returns loader from tensor dataset from

    может создать датасет у которого всего одна проекция взята за таргет из уже существующего датасета или по пути на файл:
    take_one_projection_for_data - указываем в этом параметре номер координатной проекции (нумерация от 1)
    '''
    if path:
        N = int(path.split('_')[0])
        K = int(path.split('_')[-1].split('.')[0])

        dataset = torch.load(str(N) + '_dataset_K_' + str(K) + '.pt')
        dataset = [(elem[0], elem[1]) for elem in dataset]

        if take_one_projection_for_data:
 
            dataset = [(elem[0], elem[-1][take_one_projection_for_data - 1].unsqueeze(dim=0)) for elem in dataset]

        train_data, val_data = train_test_split(dataset, test_size=0.33, random_state=42)
        if cut_size:
            train_data = train_data[:cut_size]
            val_data = val_data[:cut_size]
        train_dataloader, val_dataloader = create_dataloaders(train_data, val_data)
        return train_data, val_data, train_dataloader, val_dataloader
    
    X = []
    Y = []

    for _ in range(500):
        # X = (torch.rand(1)).squeeze().unsqueeze(dim=0)
        x = (torch.rand(K))
        y = function(x)
        X.append(x)
        Y.append(y)

    X_train, X_val, Y_train, Y_val = train_test_split(X, Y, random_state=42, train_size=0.8)

    X_train = torch.stack(X_train)
    X_val = torch.stack(X_val)
    Y_train = torch.stack(Y_train)
    Y_val = torch.stack(Y_val)

    train_data = TensorDataset(X_train, Y_train)
    val_data = TensorDataset(X_val, Y_val)

    train_dataloader = DataLoader(train_data, batch_size=128)
    val_dataloader = DataLoader(val_data, batch_size=128)

    return train_data, val_data, train_dataloader, val_dataloader
    
train_data, val_data, train_dataloader, val_dataloader = recieve_loaders(take_one_projection_for_data=1, path=path, cut_size=100)

Сама частица, отступаем от нее вектор силы

In [119]:
# N=2, K=1:

# flattened_size = CFG.K ** 2
# output_size = CFG.K

# model = nn.Sequential(
#     nn.Linear(flattened_size, 8),
#     nn.ReLU(),
#     # nn.Dropout(0.3),
#     nn.BatchNorm1d(8),

#     nn.Linear(8, 32),
#     nn.ReLU(),
#     # nn.Dropout(0.3),
#     nn.BatchNorm1d(32),

#     nn.Linear(32, 128),
#     nn.ReLU(),
#     # nn.Dropout(0.3),
#     nn.BatchNorm1d(128),
#     nn.Linear(128, output_size),
#     ).to(CFG.device)

# optimizer = optim.Adam(model.parameters(), lr=5e-3, betas=(0.9, 0.999), weight_decay=0.01)

# exp_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

In [162]:
model = SingleNet(
    output_size=1,

    # activation=nn.Sigmoid(),
    activation=nn.ReLU(),

).to(CFG.device)

# model = nn.Linear(1, 1)

# model = nn.Sequential(
#     nn.Linear(CFG.K ** 2, 8),
#     nn.ReLU(),
#     # nn.Dropout(0.5),
#     # nn.BatchNorm1d(8),

#     nn.Linear(8, 32),
#     nn.ReLU(),
#     # nn.Dropout(0.5),
#     # nn.BatchNorm1d(32),

#     nn.Linear(32, CFG.K)
# )

optimizer = optim.Adam(model.parameters(), lr=5e-3, betas=(0.9, 0.999), weight_decay=0.01)

# scheduler.step нужно первый раз делать обязательно после optimizer.step, потому что иначе мы просто пропустим первый шаг scheduler
exp_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.95)

model

SingleNet(
  (FC): Sequential(
    (0): Linear(in_features=9, out_features=64, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=64, out_features=128, bias=True)
    (4): ReLU()
    (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=128, out_features=256, bias=True)
    (7): ReLU()
    (8): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Linear(in_features=256, out_features=1, bias=True)
  )
)

In [163]:
history = train(
    train_loader=train_dataloader, val_loader=val_dataloader, model=model, optimizer=optimizer,
    
    scheduler=exp_scheduler,

    scaler=None,
    
    # criterion=MultiOutputMSELoss(),     # on K=1 it's the same as nn.MSELoss
    criterion=nn.MSELoss(),
    # criterion = GaussianNLLLossWithReadyVar(),
    
    epochs=50
)

epoch:  32%|███▏      | 16/50 [00:00<00:00, 102.15it/s]


Epoch 001 train_loss: 2.4729 val_loss 1.0686

Epoch 002 train_loss: 1.5410 val_loss 1.0186

Epoch 003 train_loss: 1.0262 val_loss 0.9718

Epoch 004 train_loss: 0.8049 val_loss 0.9150

Epoch 005 train_loss: 0.6770 val_loss 0.8558

Epoch 006 train_loss: 0.5576 val_loss 0.8199

Epoch 007 train_loss: 0.5582 val_loss 0.8064

Epoch 008 train_loss: 0.4875 val_loss 0.7975

Epoch 009 train_loss: 0.4120 val_loss 0.7720

Epoch 010 train_loss: 0.3604 val_loss 0.7495

Epoch 011 train_loss: 0.2937 val_loss 0.7351

Epoch 012 train_loss: 0.2911 val_loss 0.7316

Epoch 013 train_loss: 0.2572 val_loss 0.7402

Epoch 014 train_loss: 0.2759 val_loss 0.7470

Epoch 015 train_loss: 0.2553 val_loss 0.7466

Epoch 016 train_loss: 0.2465 val_loss 0.7507


epoch:  78%|███████▊  | 39/50 [00:00<00:00, 95.18it/s] 


Epoch 017 train_loss: 0.2772 val_loss 0.7556

Epoch 018 train_loss: 0.2029 val_loss 0.7673

Epoch 019 train_loss: 0.1885 val_loss 0.7854

Epoch 020 train_loss: 0.1845 val_loss 0.7992

Epoch 021 train_loss: 0.2064 val_loss 0.8041

Epoch 022 train_loss: 0.1909 val_loss 0.8098

Epoch 023 train_loss: 0.1631 val_loss 0.8346

Epoch 024 train_loss: 0.1603 val_loss 0.8410

Epoch 025 train_loss: 0.1471 val_loss 0.8494

Epoch 026 train_loss: 0.1751 val_loss 0.8505

Epoch 027 train_loss: 0.1394 val_loss 0.8517

Epoch 028 train_loss: 0.1689 val_loss 0.8654

Epoch 029 train_loss: 0.1796 val_loss 0.8860

Epoch 030 train_loss: 0.1416 val_loss 0.9048

Epoch 031 train_loss: 0.1364 val_loss 0.9129

Epoch 032 train_loss: 0.1317 val_loss 0.9187

Epoch 033 train_loss: 0.1332 val_loss 0.9401

Epoch 034 train_loss: 0.1168 val_loss 0.9676

Epoch 035 train_loss: 0.2215 val_loss 0.9984

Epoch 036 train_loss: 0.1295 val_loss 1.0412

Epoch 037 train_loss: 0.1646 val_loss 1.0389

Epoch 038 train_loss: 0.1607 val_

epoch: 100%|██████████| 50/50 [00:00<00:00, 98.29it/s] 


Epoch 040 train_loss: 0.1118 val_loss 1.0051

Epoch 041 train_loss: 0.1096 val_loss 1.0693

Epoch 042 train_loss: 0.0936 val_loss 1.0848

Epoch 043 train_loss: 0.0872 val_loss 1.0443

Epoch 044 train_loss: 0.0977 val_loss 0.9955

Epoch 045 train_loss: 0.0722 val_loss 0.9719

Epoch 046 train_loss: 0.1057 val_loss 0.9995

Epoch 047 train_loss: 0.1470 val_loss 1.0372

Epoch 048 train_loss: 0.1532 val_loss 1.0572

Epoch 049 train_loss: 0.0855 val_loss 1.0230

Epoch 050 train_loss: 0.1053 val_loss 1.0084


In [164]:
make_predictions_and_plot(model=model, data=train_data, same_axis=True)

Metric: 0.10423701256513596
